<a href="https://colab.research.google.com/github/Bjorn455215/B103021055/blob/main/EX05_02_Ask_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q line-bot-sdk #-q 不顯示太多文字

In [5]:
!pip install -q flask pyngrok flask_ngrok

In [10]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTOTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)


from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('Line_channel_accesstoken'))
handler = WebhookHandler(userdata.get('Line_channel_token'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

import google.generativeai as genai #設置 Google Gemini 模型
#配置 Google Gemini 模型，並定義了一個 ask_gemini 函數，用於向模型提問并获取答案。
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel('gemini-2.0-flash')
def ask_gemini(question):
  reponse = model.generate_content(question)
  return reponse.text

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        reponse = ask_gemini(event.message.text) #它會將用戶發送的訊息傳遞給 ask_gemini 函數，获取 Gemini 模型生成的回覆。
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    TextMessage(text=reponse) #藉由使用者輸入的訊息 來給出相對應的回應
                ]
            )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://978e-34-75-164-70.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [06/May/2025 07:55:50] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 07:55:50] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 07:56:05] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 07:56:20] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 07:56:56] "POST /callback HTTP/1.1" 200 -
